In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import joblib

# 1. Load and Clean Dataset
df = pd.read_csv('keypoints.csv', header=0)
df = df.dropna()

# Convert all values to float32 except label
X = df.iloc[:, :-1].astype(np.float32)
y = df.iloc[:, -1].astype(str)

# Normalize keypoints
def normalize(row):
    coords = row.values.reshape(-1, 2)
    coords -= coords[0]  # Make first keypoint the origin
    max_val = np.max(np.abs(coords))
    return (coords / max_val).flatten() if max_val != 0 else coords.flatten()

X_normalized = X.apply(normalize, axis=1, result_type='expand')

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Save label encoder
joblib.dump(le, 'label_encoder.pkl')

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X_normalized, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)

# Compute class weights
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_encoded), y=y_encoded
)
class_weights_dict = dict(enumerate(class_weights))

# 2. Build Improved Model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(y_cat.shape[1], activation='softmax'))

# 3. Compile Model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 4. Train Model
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

# Save model
model.save('final_model.h5')


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1684 - loss: 3.5583

28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 67ms/step - accuracy: 0.1701 - loss: 3.5422 - val_accuracy: 0.1050 - val_loss: 2.6999
Epoch 2/100
18/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4331 - loss: 2.1504 

28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.4450 - loss: 2.0938 - val_accuracy: 0.2055 - val_loss: 2.6667
Epoch 3/100
25/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5858 - loss: 1.5985

28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5912 - loss: 1.5876 - val_accuracy: 0.0913 - val_loss: 2.6434
Epoch 4/100
22/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7400 - loss: 1.1811 

...

28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9902 - loss: 0.2863 - val_accuracy: 1.0000 - val_loss: 0.2575
Epoch 48/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9981 - loss: 0.2739 - val_accuracy: 0.9452 - val_loss: 0.3865
Epoch 49/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9901 - loss: 0.2814 - val_accuracy: 0.9954 - val_loss: 0.2811
Epoch 50/100
19/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9955 - loss: 0.2679

28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9962 - loss: 0.2674 - val_accuracy: 1.0000 - val_loss: 0.2492
Epoch 51/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9912 - loss: 0.2628 - val_accuracy: 1.0000 - val_loss: 0.2551
Epoch 52/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9957 - loss: 0.2558 - val_accuracy: 0.7671 - val_loss: 1.0588
Epoch 53/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.9922 - loss: 0.2737 - val_accuracy: 0.7854 - val_loss: 0.9240
Epoch 54/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9985 - loss: 0.2508 - val_accuracy: 0.9680 - val_loss: 0.3344
Epoch 55/100
21/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9916 - loss: 0.2582

28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9925 - loss: 0.2571 - val_accuracy: 0.9909 - val_loss: 0.2489
Epoch 56/100
19/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2484 

28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9995 - loss: 0.2491 - val_accuracy: 0.9909 - val_loss: 0.2403
Epoch 57/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9989 - loss: 0.2486 - val_accuracy: 0.9909 - val_loss: 0.2872
Epoch 58/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9936 - loss: 0.2671 - val_accuracy: 0.9954 - val_loss: 0.2450
Epoch 59/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9973 - loss: 0.2449 - val_accuracy: 0.9954 - val_loss: 0.2734
Epoch 60/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9991 - loss: 0.2423 - val_accuracy: 0.9909 - val_loss: 0.2480
Epoch 61/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9965 - loss: 0.2529 - val_accuracy: 0.9087 - val_loss: 0.4129
Epoch 62/100
19/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2334

28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 1.0000 - loss: 0.2335 - val_accuracy: 1.0000 - val_loss: 0.2243
Epoch 63/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.9988 - loss: 0.2338 - val_accuracy: 0.9909 - val_loss: 0.2536
Epoch 64/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9956 - loss: 0.2409 - val_accuracy: 0.9863 - val_loss: 0.2605
Epoch 65/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9986 - loss: 0.2351 - val_accuracy: 0.8904 - val_loss: 0.4697
Epoch 66/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9969 - loss: 0.2334 - val_accuracy: 0.7900 - val_loss: 1.2746
Epoch 67/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9993 - loss: 0.2258 - val_accuracy: 0.8447 - val_loss: 0.5981
Epoch 68/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9984 - loss: 0.2313 - val_accuracy: 0.8447 - val_loss: 0.7642
Epoch 69/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9918 - loss: 0.2321 - val_accuracy: 0.9361 